<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/noise_and_without_noise_db_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Okay , now it's time to add noise to the database to implement local differential privacy

In [0]:
import torch

In [0]:
#creating a random database filled with 1'0 and 0's 
db = torch.rand(100) > 0.5


In [3]:
db

tensor([0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1,
        0, 1, 0, 0], dtype=torch.uint8)

In [17]:
torch.mean(db.float())  #chekcing the mean query over original database created just now

tensor(0.5019)

Now to add noise , we use coin - toss algorithm to give plausible deniabllility to the person involved in any kind of survey which we will be doing over some taboo behaviour.

The algorithm is like - 
the respondent has to answer according to following rules - 
* First they flip a coin , if head comes up , they answer honestly but if tails come up , they answer according to second flip.
* so , it means , now second flip will determine so , if now heads come up  , then it's yes and if tails then no , that means dishonest answers.

This gives the individual plausible deniability that if they are asked about something that they have said yes for and that was crime then they can say that they just said that due to random coin outcome and not really from heart.

In [0]:
def query(db):
  true_stat = torch.mean(db.float())  #query result over original database created
  first_flip = torch.rand(len(db)) > 0.5  #creating a new tensor for first flip of a coin 1 -> head , 0-> tail 
  second_flip = torch.rand(len(db)) > 0.5 #creating a new tensor for second flip of a coin 
  rdb = [] #empty list
  for i in range(len(db)):
    if(first_flip[i] == 1): 
      rdb.append(db[i])
    else:
      rdb.append(second_flip[i])
  noise_db = torch.tensor(rdb) #noised database
  #augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
  aug_true_db = torch.mean(noise_db.float())*2 - 0.5 #calulcaintg real statistics removing noise 
  return true_stat, aug_true_db
  

In [0]:
true, random = query(db) 

In [6]:
print("without noise: " + str(true))
print("with noise: " + str(random))

without noise: tensor(0.5000)
with noise: tensor(0.6600)


As expected , the true mean and mean from noised database , differs vastly thus affecting accuracy , but preserving diifferential privacy.
The vast difference that we see , is only true for small datasets, if we have huge datasets with large number of entries , we will see in few moment , this difference is very little.

In [0]:
db = torch.rand(1000) > 0.5


In [0]:
true, random = query(db)

In [9]:
print("without noise: " + str(true))
print("with noise: " + str(random))

without noise: tensor(0.5050)
with noise: tensor(0.5300)


In [0]:
db = torch.rand(10000) > 0.5


In [0]:
true, random = query(db)

In [12]:
print("without noise: " + str(true))
print("with noise: " + str(random))

without noise: tensor(0.4958)
with noise: tensor(0.4968)


In [0]:
db = torch.rand(100000) > 0.5


In [0]:
true, random = query(db)

In [15]:
print("without noise: " + str(true))
print("with noise: " + str(random))

without noise: tensor(0.5019)
with noise: tensor(0.4973)
